In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [2]:
url='https://www.flipkart.com/search?q=mobile+phones+under+30k&as=on&as-show=on&otracker=AS_Query_OrganicAutoSuggest_5_20_na_na_na&otracker1=AS_Query_OrganicAutoSuggest_5_20_na_na_na&as-pos=5&as-type=RECENT&suggestionId=mobile+phones+under+30k&requestId=da5d64a3-d3b9-4343-b89c-f8fed34abf98&as-searchtext=mobile%20phones%20under%2050'

In [3]:
req = requests.get(url)
req

<Response [200]>

In [4]:
soup = BeautifulSoup(req.text,'lxml')
link = soup.find('a',class_= '_1LKTO3').get('href')
# to build the complete link we need to use flipkart.com
c_link = 'https://www.flipkart.com'+link

In [16]:
page =0
mobile = []
price = []
dict = {'Mobile':mobile,'Price':price}

while page<25:
    url = c_link[:-1]+str(page)
    req = requests.get(url)
    soup = BeautifulSoup(req.text,'lxml')
      
    name = soup.find_all('div', class_="_4rR01T")
    for i in name:
        dict["Mobile"].append(i.text)
        
    prices = soup.find_all('div', class_="_30jeq3 _1_WHN1")
    for i in prices:
        # we need to convert the string '26,999' to 26999(int)
        dict["Price"].append(int(i.text[1:].replace(",","")))

    page = page+1

    
df = pd.DataFrame(dict)
df.head(10)

,Mobile,Price
0,"SAMSUNG Galaxy M33 5G (Deep Ocean Blue, 128 GB)",17400
1,"SAMSUNG Galaxy M33 5G (Mystique Green, 128 GB)",17379
2,"REDMI Note 12 Pro 5G (Stardust Purple, 128 GB)",23999
3,"SAMSUNG Galaxy S20 FE 5G (Cloud Navy, 128 GB)",28464
4,"REDMI Note 12 Pro 5G (Stardust Purple, 256 GB)",26999
5,"REDMI Note 12 Pro+ 5G (Obsidian Black, 256 GB)",29999
6,"realme 10 Pro+ 5G (Nebula Blue, 128 GB)",25999
7,"realme 10 Pro+ 5G (Hyperspace, 128 GB)",25999
8,"REDMI Note 12 Pro+ 5G (Arctic White, 256 GB)",29999
9,"REDMI Note 12 Pro 5G (Stardust Purple, 128 GB)",24999


In [17]:
df.duplicated().value_counts()

False    506
True      94
dtype: int64

In [18]:
df.drop_duplicates(inplace=True)

In [19]:
df.head(5)

,Mobile,Price
0,"SAMSUNG Galaxy M33 5G (Deep Ocean Blue, 128 GB)",17400
1,"SAMSUNG Galaxy M33 5G (Mystique Green, 128 GB)",17379
2,"REDMI Note 12 Pro 5G (Stardust Purple, 128 GB)",23999
3,"SAMSUNG Galaxy S20 FE 5G (Cloud Navy, 128 GB)",28464
4,"REDMI Note 12 Pro 5G (Stardust Purple, 256 GB)",26999


In [20]:
# lets remove the features from the mobile names
def preprocess(data):
    l=[]
    for i in data:
        if i == '(':
            break
        else :
            l.append(i)
    return ''.join(l)

In [21]:
df['Mobile'] = df['Mobile'].apply(lambda x: preprocess(x))

In [22]:
df.head(10)

,Mobile,Price
0,SAMSUNG Galaxy M33 5G,17400
1,SAMSUNG Galaxy M33 5G,17379
2,REDMI Note 12 Pro 5G,23999
3,SAMSUNG Galaxy S20 FE 5G,28464
4,REDMI Note 12 Pro 5G,26999
5,REDMI Note 12 Pro+ 5G,29999
6,realme 10 Pro+ 5G,25999
7,realme 10 Pro+ 5G,25999
8,REDMI Note 12 Pro+ 5G,29999
9,REDMI Note 12 Pro 5G,24999


In [23]:
df.to_csv("mobile_data.csv")